### Imports

In [103]:
import requests
from xml.etree import ElementTree as ET
from Bio.PDB import PDBParser
from Bio.PDB.SASA import ShrakeRupley
import pandas as pd
import os
from Bio.ExPASy import ScanProsite
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import csv
import random

In [2]:
# GIVEN A PDB ID RETURN THE UNIPROT ID


def pdb_to_uniprot(pdb_id):
    # Define the API endpoint for the specific PDB ID
    url = f"https://www.ebi.ac.uk/pdbe/api/mappings/uniprot/{pdb_id}"
    
    # Send the request to the PDBe API
    response = requests.get(url)
    if response.status_code == 404:
        print(f"PDB ID {pdb_id} not found (HTTP Error 404).")
        return None
    
    content = response.json()
    
    # Extract UniProt IDs
    uniprot_ids = []
    if pdb_id.lower() in content:
        for uniprot in content[pdb_id.lower()]['UniProt'].keys():
            uniprot_ids.append(uniprot)
    
    return uniprot_ids

In [106]:
# Functions Needed 

# General Info 
def get_protein_info(uniprot_id):
    # Define the API endpoint for the specific UniProt ID
    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.json"
    
    # Send the request to the UniProt API
    response = requests.get(url)
    data = response.json()
    return data


#Get the sequence Given Uniprot
def get_protein_sequence(uniprot_id):
    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.json"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve sequence for UniProt ID {uniprot_id}")
        return 'N/A'

    data = response.json()
    sequence = data.get('sequence', {}).get('value', 'N/A')
    return sequence


#Get the EC NUMBERS
def extract_ec_numbers(protein_info):
    ec_numbers = []
    for comment in protein_info.get('comments', []):
        if comment.get('commentType') == 'CATALYTIC ACTIVITY':
            reaction = comment.get('reaction', {})
            ec_number = reaction.get('ecNumber')
            if ec_number:
                ec_numbers.append(ec_number)
    return ec_numbers

#downloads PDB XML FILE (EASIER TO PARSE WITH MACHINE)
def get_pdbml(pdb_id):
    url = f"https://files.rcsb.org/view/{pdb_id}.xml"
    response = requests.get(url)
    return response.content


#find SECONDARY STRUCTURES
def parse_secondary_structure(pdbml_content):
    root = ET.fromstring(pdbml_content)
    ns = {'pdbx': 'http://pdbml.pdb.org/schema/pdbx-v50.xsd'}
    
    sec_struct = []
    for ss in root.findall('.//pdbx:struct_conf', ns):
        start_res = ss.find('pdbx:beg_auth_seq_id', ns).text
        end_res = ss.find('pdbx:end_auth_seq_id', ns).text
        conf_type = ss.find('pdbx:conf_type_id', ns).text
        sec_struct.append((start_res, end_res, conf_type))
    
    for ss in root.findall('.//pdbx:struct_sheet_range', ns):
        start_res = ss.find('pdbx:beg_auth_seq_id', ns).text
        end_res = ss.find('pdbx:end_auth_seq_id', ns).text
        sec_struct.append((start_res, end_res, 'STRAND'))
    
    return sec_struct

#get the secondary Structures
def get_secondary_structure(pdb_id):
    pdbml_content = get_pdbml(pdb_id)
    secondary_structure = parse_secondary_structure(pdbml_content)
    return secondary_structure

#Get surface area
def Get_SA(pdb_id):
    # Define the URL to download the PDB file
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    file_path = f"/Users/shayneskrtic/Desktop/{pdb_id}.pdb"
    
    # Download the PDB file
    response = requests.get(url)
    with open(file_path, 'wb') as file:
        file.write(response.content)
    
    # Parse the PDB file and calculate SASA
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure(pdb_id, file_path)
    sr = ShrakeRupley()
    sr.compute(structure, level="S")
    return int(structure.sasa)
    
    # Delete the PDB file after calculation
    os.remove(file_path)

#Motif finder? Bad?
def search_motifs(sequence):
    # Save the sequence to a temporary file
    with open("temp.fasta", "w") as file:
        file.write(f">query\n{sequence}\n")
    
    # Read the sequence from the file
    record = SeqIO.read("temp.fasta", "fasta")
    
    # Perform the motif search using ScanProsite
    handle = ScanProsite.scan(seq=record.seq)
    results = ScanProsite.read(handle)
    
    # Print the results
    return results
    #for result in results:
        #print(f"Motif: {result['signature_ac']}, Start: {result['start']}, End: {result['stop']}")
    
    # Clean up the temporary file
    os.remove("temp.fasta")

#AA composition block 
def amino_acid_composition(sequence):
    # Initialize a dictionary to store the counts of each amino acid
    amino_acid_counts = {
        'A': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 0, 'I': 0,
        'K': 0, 'L': 0, 'M': 0, 'N': 0, 'P': 0, 'Q': 0, 'R': 0, 'S': 0,
        'T': 0, 'V': 0, 'W': 0, 'Y': 0
    }
    
    # Count the occurrences of each amino acid in the sequence
    for amino_acid in sequence:
        if amino_acid in amino_acid_counts:
            amino_acid_counts[amino_acid] += 1
    
    # Calculate the frequency of each amino acid
    sequence_length = len(sequence)
    amino_acid_composition = {aa: (count / sequence_length)* 100 for aa, count in amino_acid_counts.items()}
    
    return amino_acid_composition

# Properties 
def calculate_physicochemical_properties(sequence):
    # Create a ProteinAnalysis object
    analysed_seq = ProteinAnalysis(sequence)
    
    # Calculate molecular weight
    molecular_weight = analysed_seq.molecular_weight()
    
    # Calculate hydrophobicity (Kyte-Doolittle scale)
    hydrophobicity = analysed_seq.gravy()
    
    return molecular_weight, hydrophobicity


#Do all three and Print Nicely
def do_all(pdb_id):
    uniprot_id = pdb_to_uniprot(pdb_id)[0]
    protein_info = get_protein_info(uniprot_id)
    if not protein_info:
        return None
    
    # Extract relevant information from the protein_info JSON
    protein_name = protein_info.get('proteinDescription', {}).get('recommendedName', {}).get('fullName', {}).get('value', 'N/A')
    organism = protein_info.get('organism', {}).get('scientificName', 'N/A')
    gene_name = protein_info.get('genes', [{}])[0].get('geneName', {}).get('value', 'N/A')
    ec_numbers = extract_ec_numbers(protein_info)
    sequence = get_protein_sequence(uniprot_id)
    secondary_structure = get_secondary_structure(pdb_id)
    Surface_area = Get_SA(pdb_id)
    sequence_length = len(sequence)
    SA_div_Sequence_len = Surface_area/sequence_length
    motifs = search_motifs(sequence)
    num_of_helicies = sum(1 for structure in secondary_structure if structure[2] == "HELX_P")
    num_of_strand = sum(1 for structure in secondary_structure if structure[2] == "STRAND")
    num_of_helicies = sum(1 for structure in secondary_structure if structure[2] == "HELX_P")
    total_helix_length = sum(int(structure[1]) - int(structure[0]) + 1 for structure in secondary_structure if structure[2] == "HELX_P")
    total_strand_length = sum(int(structure[1]) - int(structure[0]) + 1 for structure in secondary_structure if structure[2] == "STRAND")
    percent_helix = round(float((total_helix_length / sequence_length) * 100),2)
    percent_strand = round(float((total_strand_length / sequence_length) * 100),2)
    percent_else = round(100 - (percent_helix + percent_strand),2)
    longest_helix_length = max((int(structure[1]) - int(structure[0]) + 1 for structure in secondary_structure if structure[2] == "HELX_P"), default=0)
    longest_strand_length = max((int(structure[1]) - int(structure[0]) + 1 for structure in secondary_structure if structure[2] == "STRAND"), default=0)
    amino_acid_count = amino_acid_composition(sequence) 
    Molecular_weight, Hydrophobicity = calculate_physicochemical_properties(sequence)
    
    


    
    return {
        'pdb_id' : pdb_id,
        'protein_name': protein_name,
        'Uniprot_Id':uniprot_id,
        'organism': organism,
        'gene_name': gene_name,
        'ec_numbers': ec_numbers,
        'sequence': sequence,
        'secondary_structure' : secondary_structure,
        'Surface_area' : Surface_area,
        'sequence_length' : sequence_length,
        'SA_div_Sequence_len' : SA_div_Sequence_len,
        'motifs' : motifs,
        'num_of_helicies' : num_of_helicies,
        'num_of_strand' : num_of_strand,
        'total_helix_length' : total_helix_length,
        'total_Strand_length' : total_strand_length,
        'percent_helix' : percent_helix,
        'percent_strand' : percent_strand,
        'percent_else' : percent_else,
        'longest_helix_length' : longest_helix_length,
        'longest_strand_length' : longest_strand_length,
        'amino_acid_Percent' : amino_acid_count,
        'Molecular_weight' : Molecular_weight,
        'Hydrophobicity' : Hydrophobicity
        
        
    }

In [132]:
#Test PDB LIST

pdbids = ['1V48']

In [104]:
#make and add to CSV
def append_to_csv(data, csv_file):
    # Define the header based on the keys of the data dictionary
    header = list(data.keys())
    
    # Check if the CSV file exists and if it has a header
    try:
        with open(csv_file, 'r') as file:
            reader = csv.reader(file)
            existing_header = next(reader)
    except FileNotFoundError:
        existing_header = None
    
    # Open the CSV file in append mode
    with open(csv_file, 'a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=header)
        
        # Write the header if it doesn't exist
        if existing_header != header:
            writer.writeheader()
        
        # Write the data
        writer.writerow(data)

In [133]:
#Test append with small num of pdb_ids

for id in pdbids:
    #Given a pdb id, read csv file append it to csv file dictionary:feature
    append_to_csv(do_all(id),'/Users/shayneskrtic/Desktop/data.csv')

In [117]:
import pandas as pd

def clean_csv(input_file, output_file):
    try:
        # Read the CSV file into a DataFrame, specifying the delimiter if necessary
        df = pd.read_csv(input_file, delimiter=',', on_bad_lines='skip')
        
        # Drop rows where the first column has more than three characters
        df = df[df.iloc[:, 0].str.len() <= 4]
        
        # Drop duplicate rows
        df = df.drop_duplicates()
        
        # Write the cleaned DataFrame back to a CSV file
        df.to_csv(output_file, index=False)
        
        print(f"Cleaned CSV file saved to {output_file}")
    
    except pd.errors.ParserError as e:
        print(f"ParserError: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Example usage
input_file = '/Users/shayneskrtic/Desktop/output.csv'
output_file = '/Users/shayneskrtic/Desktop/output_cleaned.csv'
clean_csv(input_file, output_file)


Cleaned CSV file saved to /Users/shayneskrtic/Desktop/output_cleaned.csv


In [120]:
def append_ids_to_df(input_file, df=None):
    # Read the CSV file into a DataFrame
    csv_df = pd.read_csv(input_file)
    
    # Extract the IDs from the first column
    ids = csv_df.iloc[:, 0]
    
    # If no DataFrame is provided, create a new one
    if df is None:
        df = pd.DataFrame(columns=['ID'])
    
    # Append the IDs to the DataFrame using concat
    df = pd.concat([df, pd.DataFrame(ids, columns=['ID'])], ignore_index=True)
    
    return df

# Example usage
input_file = '/Users/shayneskrtic/Desktop/output_cleaned.csv'
df = append_ids_to_df(input_file)
print(df)

        ID
0     6Y7R
1     3WZK
2     2XVM
3     2I2B
4     5IBO
...    ...
4793  8WMD
4794  6NPI
4795  2F8E
4796  5W07
4797     }

[4798 rows x 1 columns]


In [122]:
def select_random_rows(df, num_rows=2000):
    # Ensure the DataFrame has at least num_rows rows
    if len(df) < num_rows:
        raise ValueError(f"The DataFrame has only {len(df)} rows, which is less than the requested {num_rows} rows.")
    
    # Select num_rows random rows from the DataFrame
    random_df = df.sample(n=num_rows, random_state=1)
    
    return random_df

In [125]:
final_2000_df = (select_random_rows(df))

In [126]:
print(final_2000_df)

        ID
3835  1V48
1356  4BUT
3499  7MRR
552   4ON3
3766  4HL5
...    ...
1502  7A5M
2013  4ED0
2390  5D6E
1301  3WZJ
4313  4EHY

[2000 rows x 1 columns]


In [162]:
def make_the_dataset(df):
    int = 1683
    for id in df['ID'][1725:]:
        try:
            # Given a pdb id, read csv file and append it to csv file dictionary:feature
            append_to_csv(do_all(id), '/Users/shayneskrtic/Desktop/data.csv')
            int = int + 1
            print(f'{id} done {int}/2798 or {(int/2798)*100:.2f} %')
            
        except Exception as e:
            print(f'Error processing {id}: {e}')
            continue
    

In [163]:
make_the_dataset(df_3)

7L6R done 1684/2798 or 60.19 %
1N9N done 1685/2798 or 60.22 %
5A7B done 1686/2798 or 60.26 %
5RYH done 1687/2798 or 60.29 %
3PLI done 1688/2798 or 60.33 %
6JHC done 1689/2798 or 60.36 %
6G89 done 1690/2798 or 60.40 %
1A54 done 1691/2798 or 60.44 %
1WLJ done 1692/2798 or 60.47 %
4L0I done 1693/2798 or 60.51 %
8V7E done 1694/2798 or 60.54 %
PDB ID 7D8C not found (HTTP Error 404).
Error processing 7D8C: 'NoneType' object is not subscriptable
6P1M done 1695/2798 or 60.58 %
8UJV done 1696/2798 or 60.61 %
8QIU done 1697/2798 or 60.65 %
7S75 done 1698/2798 or 60.69 %
8QIK done 1699/2798 or 60.72 %
2FA1 done 1700/2798 or 60.76 %
1O2L done 1701/2798 or 60.79 %
1PPC done 1702/2798 or 60.83 %
5J54 done 1703/2798 or 60.86 %
4YYV done 1704/2798 or 60.90 %
3A25 done 1705/2798 or 60.94 %
2Z3L done 1706/2798 or 60.97 %
5RFE done 1707/2798 or 61.01 %
3AUX done 1708/2798 or 61.04 %
4P6E done 1709/2798 or 61.08 %
5RXI done 1710/2798 or 61.12 %
8GD5 done 1711/2798 or 61.15 %
2VBX done 1712/2798 or 61.19 %

In [146]:
def get_difference(df1, df2):
    # Merge df1 and df2 on all columns and use indicator to identify rows that are only in df1
    diff_df = df1.merge(df2, how='left', indicator=True)
    
    # Filter rows that are only in df1
    diff_df = diff_df[diff_df['_merge'] == 'left_only']
    
    # Drop the indicator column
    diff_df = diff_df.drop(columns=['_merge'])
    
    return diff_df

In [147]:
df_3 = get_difference(df, final_2000_df)

In [151]:
print(df_3)

        ID
0     6Y7R
2     2XVM
8     1FM4
9     2ZJM
10    3TKA
...    ...
4789  6JEX
4790  4UCO
4792  2BR0
4793  8WMD
4796  5W07

[2798 rows x 1 columns]


In [152]:
print(df)

        ID
0     6Y7R
1     3WZK
2     2XVM
3     2I2B
4     5IBO
...    ...
4793  8WMD
4794  6NPI
4795  2F8E
4796  5W07
4797     }

[4798 rows x 1 columns]


In [156]:
print(final_2000_df)

        ID
3835  1V48
1356  4BUT
3499  7MRR
552   4ON3
3766  4HL5
...    ...
1502  7A5M
2013  4ED0
2390  5D6E
1301  3WZJ
4313  4EHY

[2000 rows x 1 columns]
